##Initial Communications With Spotify API

In [21]:
import requests
import json
import yaml
import spotipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


with open('keys.json') as keys_file:
    keys = json.load(keys_file)
    keys_file.close()

In [5]:
def play_track_by_uri(uri):
    from IPython.core.display import HTML
    return HTML('<iframe src="https://embed.spotify.com/?uri=' + uri + '" width="300" height="80" frameborder="0" allowtransparency="true"></iframe>')
play_track_by_uri("spotify:artist:4Z8W4fKeB5YxbusRsdQVPb")

In [11]:
import spotipy.util
scope = 'user-library-read playlist-read-private user-read-private'
token = spotipy.util.prompt_for_user_token(
    'dbgoldberg01', scope,
    client_id=keys['CLIENT_ID'],
    client_secret=keys['CLIENT_SECRET'],
    redirect_uri='http://localhost:8888/callback')
if token:
    sp = spotipy.Spotify(auth=token)
else:
    print "Can't get token"
print json.dumps(sp.me(),indent=2)

{
  "product": "premium", 
  "display_name": "David Goldberg", 
  "external_urls": {
    "spotify": "https://open.spotify.com/user/124292901"
  }, 
  "country": "US", 
  "uri": "spotify:user:124292901", 
  "href": "https://api.spotify.com/v1/users/124292901", 
  "followers": {
    "total": null, 
    "href": null
  }, 
  "images": [
    {
      "url": "https://fbcdn-profile-a.akamaihd.net/hprofile-ak-xfp1/v/t1.0-1/s200x200/8809_10201450874662610_61631215_n.jpg?oh=b052f6ad27c4ffc0a094aca5a839fb37&oe=569C036C&__gda__=1450188145_bc39634c5166d7c7755be7255344ab72", 
      "width": null, 
      "height": null
    }
  ], 
  "type": "user", 
  "id": "124292901"
}


In [12]:
me_id = "124292901"

##Exploring My "Favorites" Playlist

In [211]:
def pprint(d):
    print json.dumps(d, indent=2)

This is the data I want in my dataframe and the structure:
* my_tracks list item
    * added_at
    * track
        * name
        * id
        * uri
        * album
            * name
            * id
            * uri
        * popularity
        * artists (list)
            * name
            * id
            * uri

In [17]:
def get_playlist_tracks(name):
    pls = sp.user_playlists(me_id)
    res = [p for p in pls['items']
                 if p['name']==name]
    playlist_id = res[0]['id']
    ps = sp.user_playlist_tracks(
        "124292901", playlist_id=playlist_id)
    return ps['items']
    
    
ps_tracks = get_playlist_tracks('science')

In [14]:
ps_tracks[0].keys()

[u'is_local', u'track', u'added_by', u'added_at']

In [20]:
def make_df_from_track_list(track_list):
    tracks_data = [ \
     (track['track']['name'], 
      track['track']['id'],
      track['track']['uri'],
      track['track']['popularity'],
      track['added_at'],
      track['track']['album']['name'],
      track['track']['album']['id'],
      track['track']['album']['uri'],
      track['track']['artists'][0]['name'],
      track['track']['artists'][0]['id'],
      track['track']['artists'][0]['uri'],
     ) for track in track_list]
    tracks_df = pd.DataFrame(tracks_data, columns=[\
        'track_name',
        'track_id',
        'track_uri',
        'track_popularity',
        'added_at',
        'album_name',
        'album_id',
        'album_uri',
        'artist_name',
        'artist_id',
        'artist_uri'])
    return tracks_df
ps_df = make_df_from_track_list(ps_tracks)
print ps_df.shape
ps_df.head()
    

(98, 11)


,track_name,track_id,track_uri,track_popularity,added_at,album_name,album_id,album_uri,artist_name,artist_id,artist_uri
0,Graceland,3lbRocWWpBVO9eefdjGi60,spotify:track:3lbRocWWpBVO9eefdjGi60,12,2015-09-16T22:33:32Z,Graceland (2011 Remaster),4O7d2vnCa5sfyZph6gU4sa,spotify:album:4O7d2vnCa5sfyZph6gU4sa,Paul Simon,2CvCyf1gEVhI0mX6aFXmVI,spotify:artist:2CvCyf1gEVhI0mX6aFXmVI
1,Speak In,76GGyeUiOtzsZjCxIJGDpd,spotify:track:76GGyeUiOtzsZjCxIJGDpd,0,2015-09-16T22:33:32Z,It Could Just Be This Place,14M1Is6t3MOkxjBynarKce,spotify:album:14M1Is6t3MOkxjBynarKce,not an Airplane,2eTQ0GfNG2zrAghLUKkSas,spotify:artist:2eTQ0GfNG2zrAghLUKkSas
2,The Funeral,4o0NjemqhmsYLIMwlcosvW,spotify:track:4o0NjemqhmsYLIMwlcosvW,76,2015-09-16T22:33:32Z,Everything All The Time,5uMfshtC2Jwqui0NUyUYIL,spotify:album:5uMfshtC2Jwqui0NUyUYIL,Band of Horses,0OdUWJ0sBjDrqHygGUXeCF,spotify:artist:0OdUWJ0sBjDrqHygGUXeCF
3,Open Season,6XPC7ZNTJcnWzQaLZYe3od,spotify:track:6XPC7ZNTJcnWzQaLZYe3od,43,2015-09-16T22:33:32Z,Open Season,3vyA5G1a5ISREXX0A4D2Iv,spotify:album:3vyA5G1a5ISREXX0A4D2Iv,High Highs,1cXVTtkpqSXVhyD32f9MS4,spotify:artist:1cXVTtkpqSXVhyD32f9MS4
4,Artificial Light,2Y93hToFlR6CBItrUqsGqG,spotify:track:2Y93hToFlR6CBItrUqsGqG,40,2015-09-16T22:33:32Z,White Lighter,4cPNeJTCwhYoJJ8sfqLOCB,spotify:album:4cPNeJTCwhYoJJ8sfqLOCB,Typhoon,6p6WlxIkeWH8yOYvciFfVl,spotify:artist:6p6WlxIkeWH8yOYvciFfVl


##Scraping From Echo Nest
for each track, I want to get:
* artists
    * biographies
    * discovery
    * familiarity
    * genre
    * hotttnesss

* track
    * danceability
    * duration
    * energy
    * key
    * loudness
    * mode
    * speechiness
    * acousticness
    * liveness
    * tempo
    * time_signature

**Note:** I should do artists and tracks separately so that I don't exceed the api call limit

###Adding Echo Nest Artist Data

In [148]:
def get_en_artist_data(artist_uri):
    base_url = 'http://developer.echonest.com/api/v4/artist/profile?'
    suffix = \
        'api_key='+ keys['EN_API_KEY']+'&'\
        'id='+artist_uri+'&'+\
        'bucket=genre'+'&'+\
        'bucket=biographies'+'&'+\
        'bucket=discovery'+'&'+\
        'bucket=familiarity'+'&'+\
        'bucket=hotttnesss'
    r = requests.get(base_url+suffix)
    return json.loads(r.content)['response']
k = get_en_artist_data('spotify:artist:066X20Nz7iquqkkCW6Jxy6')
print k['status']

{u'code': 0, u'message': u'Success', u'version': u'4.2'}


k
* artist
    * genres
        * list of dicts: {name: <value>}
    * biographies
        * list of bios
            * text
    * familiarity
    * hotttnesss
    * discovery

In [174]:
def get_data_from_artist_uri(artist_uri):
    artist = get_en_artist_data(artist_uri).get('artist')
    artist_data =  (\
      [g['name'] for g in artist['genres']], 
      artist['familiarity'],
      artist['hotttnesss'],
      artist['discovery'],
      [bio['text'] for bio in artist['biographies']]
    )
    return artist_data
# get_data_from_artist_uri('spotify:artist:066X20Nz7iquqkkCW6Jxy6')

In [161]:
artist_uris = ps_df['artist_uri']
artist_series_columns = ['genres','familiarity','hotttnesss',
        'discovery','biographies']
artist_data = []
import time
for i, uri in enumerate(artist_uris):
    try:
        artist_data.append((uri, get_data_from_artist_uri(uri)))
    except:
        print "error on uri",uri
    time.sleep(3)
    if i%10==0:print i


0
10
20
30
40
50
60
error on uri None
70
80
90


In [173]:
uris, data = zip(*artist_data)
a, b, c, d, e = zip(*data)
print a[:5]
tmp = pd.DataFrame({artist_series_columns[0]:a,
                    artist_series_columns[1]:b,
                    artist_series_columns[2]:c,
                    artist_series_columns[3]:d,
                    artist_series_columns[4]:e})
tmp['artist_uri'] = uris
ps_df2 = ps_df.copy()
ps_df2 = pd.merge(ps_df2, tmp, on='artist_uri')
ps_df2.head()

([u'singer-songwriter', u'folk rock', u'folk-pop', u'folk', u'traditional folk', u'mellow gold', u'folk christmas', u'roots rock', u'permanent wave', u'soft rock'], [], [u'indie pop', u'indie folk', u'stomp and holler', u'indie rock', u'chamber pop', u'folk-pop'], [u'shimmer psych', u'chillwave'], [u'dutch hip hop'])


,track_name,track_id,track_uri,track_popularity,added_at,album_name,album_id,album_uri,artist_name,artist_id,artist_uri,biographies,discovery,familiarity,genres,hotttnesss
0,Graceland,3lbRocWWpBVO9eefdjGi60,spotify:track:3lbRocWWpBVO9eefdjGi60,12,2015-09-16T22:33:32Z,Graceland (2011 Remaster),4O7d2vnCa5sfyZph6gU4sa,spotify:album:4O7d2vnCa5sfyZph6gU4sa,Paul Simon,2CvCyf1gEVhI0mX6aFXmVI,spotify:artist:2CvCyf1gEVhI0mX6aFXmVI,"[In a career dating back to the 1950s, Paul Si...",0.397473,0.773868,"[singer-songwriter, folk rock, folk-pop, folk,...",0.785033
1,Speak In,76GGyeUiOtzsZjCxIJGDpd,spotify:track:76GGyeUiOtzsZjCxIJGDpd,0,2015-09-16T22:33:32Z,It Could Just Be This Place,14M1Is6t3MOkxjBynarKce,spotify:album:14M1Is6t3MOkxjBynarKce,not an Airplane,2eTQ0GfNG2zrAghLUKkSas,spotify:artist:2eTQ0GfNG2zrAghLUKkSas,"[Henry Miller once wrote: ""When I think of all...",0.367769,0.324138,[],0.384201
2,The Funeral,4o0NjemqhmsYLIMwlcosvW,spotify:track:4o0NjemqhmsYLIMwlcosvW,76,2015-09-16T22:33:32Z,Everything All The Time,5uMfshtC2Jwqui0NUyUYIL,spotify:album:5uMfshtC2Jwqui0NUyUYIL,Band of Horses,0OdUWJ0sBjDrqHygGUXeCF,spotify:artist:0OdUWJ0sBjDrqHygGUXeCF,"[Emerging in 2004 with a blend of woodsy, midt...",0.358887,0.713456,"[indie pop, indie folk, stomp and holler, indi...",0.617639
3,Open Season,6XPC7ZNTJcnWzQaLZYe3od,spotify:track:6XPC7ZNTJcnWzQaLZYe3od,43,2015-09-16T22:33:32Z,Open Season,3vyA5G1a5ISREXX0A4D2Iv,spotify:album:3vyA5G1a5ISREXX0A4D2Iv,High Highs,1cXVTtkpqSXVhyD32f9MS4,spotify:artist:1cXVTtkpqSXVhyD32f9MS4,"[New York, New York http://www.myspace.com/hi...",0.578679,0.406136,"[shimmer psych, chillwave]",0.609620
4,Artificial Light,2Y93hToFlR6CBItrUqsGqG,spotify:track:2Y93hToFlR6CBItrUqsGqG,40,2015-09-16T22:33:32Z,White Lighter,4cPNeJTCwhYoJJ8sfqLOCB,spotify:album:4cPNeJTCwhYoJJ8sfqLOCB,Typhoon,6p6WlxIkeWH8yOYvciFfVl,spotify:artist:6p6WlxIkeWH8yOYvciFfVl,"[I battle rap on letsbeef.com, Typhoon is a na...",0.512166,0.474940,[dutch hip hop],0.593552


###Adding Echo Nest Track Data

* track
    * danceability
    * duration
    * energy
    * key
    * loudness
    * mode
    * speechiness
    * acousticness
    * liveness
    * tempo
    * time_signature

In [203]:
def get_en_track_data(track_uri):
    base_url = 'http://developer.echonest.com/api/v4/track/profile?'
    suffix = \
        'api_key='+ keys['EN_API_KEY']+'&'\
        'id='+track_uri+'&'+\
        'bucket=audio_summary'
    r = requests.get(base_url+suffix)
    return json.loads(r.content)['response']
k = get_en_track_data('spotify:track:0cs671lxX2eoDzr2KMuo3N')
print k['status']

{u'code': 0, u'message': u'Success', u'version': u'4.2'}


In [239]:
def get_data_from_track_uri(track_uri):
    track = get_en_track_data(track_uri).get('track')
    if track is not None:
        series = pd.Series(track.get('audio_summary').values(),
                          index=track.get('audio_summary').keys())
        series.drop('analysis_url', inplace=True)
        series.set_value('track_uri',track_uri)
        return series
    else:
#         print get_en_track_data(track_uri)['status']
        raise ValueError('Issue with Echo Nest API probably')
pd.DataFrame([get_data_from_track_uri('spotify:track:0cs671lxX2eoDzr2KMuo3N'),
get_data_from_track_uri('spotify:track:0cs671lxX2eoDzr2KMuo3N')])

,key,tempo,energy,liveness,speechiness,acousticness,instrumentalness,mode,time_signature,duration,loudness,valence,danceability,track_uri
0,2,124.51,0.787535,0.117132,0.029412,0.045096,0.041608,1,4,201.69333,-12.323,0.866205,0.66418,spotify:track:0cs671lxX2eoDzr2KMuo3N
1,2,124.51,0.787535,0.117132,0.029412,0.045096,0.041608,1,4,201.69333,-12.323,0.866205,0.66418,spotify:track:0cs671lxX2eoDzr2KMuo3N


In [247]:
track_series_columns = ['key','tempo','energy','liveness','speechiness',
        'acousticness','instrumentalness','mode','time_signature',
        'duration','loudness','valence','danceability']
track_uris = ps_df2['track_uri']
track_data = pd.DataFrame()
import time
for i, uri in enumerate(track_uris):
    try:
        track_data = track_data.append(get_data_from_track_uri(uri), ignore_index=True)
    except:
        print "error on uri",uri, 'index=',i
    time.sleep(2)
    if i%10==0:print i
track_data.head()

0
error on uri spotify:track:76GGyeUiOtzsZjCxIJGDpd index= 1
error on uri spotify:track:6XPC7ZNTJcnWzQaLZYe3od index= 3
error on uri spotify:track:5KFdwLvJ0Bokm0TFlSPq47 index= 8
10
error on uri spotify:track:4llDF3RuBQYj2nkjTCJY4w index= 14
error on uri spotify:track:4Ddaf5bxzGfW1Ut2UtpTUl index= 20
20
error on uri spotify:track:7c13Xlndg43PnSzpHRD11d index= 21
error on uri spotify:track:5lCJIgiqGmkc7uP6YJ0COc index= 22
error on uri spotify:track:63fKR9muo3uUwlFwlwZLG1 index= 23
error on uri spotify:track:69CKj4XCzIY89pYoiAaHwP index= 29
error on uri spotify:track:69CKj4XCzIY89pYoiAaHwP index= 30
30
error on uri spotify:track:69CKj4XCzIY89pYoiAaHwP index= 31
error on uri spotify:track:57V3qFWPxsUB3PTFQbWyDk index= 32
error on uri spotify:track:49HEwdXGHQkcApPc9YzdXZ index= 33
error on uri spotify:track:49HEwdXGHQkcApPc9YzdXZ index= 34
error on uri spotify:track:1d1uRlSEcYFmty6ihoAr84 index= 35
error on uri spotify:track:1d1uRlSEcYFmty6ihoAr84 index= 36
error on uri spotify:track:63vL5

,acousticness,danceability,duration,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_uri,valence
0,0.639247,0.729940,291.19955,0.690819,4.685153e-04,1,0.079664,-9.653,0,0.033910,117.932,4,spotify:track:3lbRocWWpBVO9eefdjGi60,0.868158
1,0.014234,0.315862,321.14667,0.768954,7.311096e-08,0,0.092790,-5.283,0,0.042186,122.507,4,spotify:track:4o0NjemqhmsYLIMwlcosvW,0.111796
2,0.218777,0.310621,335.36680,0.536411,2.085726e-01,2,0.357705,-8.108,1,0.044852,89.811,4,spotify:track:2Y93hToFlR6CBItrUqsGqG,0.358969
3,0.218777,0.310621,335.36680,0.536411,2.085726e-01,2,0.357705,-8.108,1,0.044852,89.811,4,spotify:track:2Y93hToFlR6CBItrUqsGqG,0.358969
4,0.262802,0.572902,201.44934,0.576814,8.972063e-03,7,0.109945,-7.360,1,0.039948,144.049,3,spotify:track:6gjmBovGkJBMA6XKqkSDci,0.311279


In [261]:
ps_df3 = ps_df2.copy()
ps_df3 = pd.merge(ps_df2, track_data, on='track_uri', how="outer")
print ps_df3.shape
ps_df3.drop_duplicates('track_uri',inplace=True)
print ps_df3.shape
ps_df3.head()

(297, 29)
(97, 29)


,track_name,track_id,track_uri,track_popularity,added_at,album_name,album_id,album_uri,artist_name,artist_id,...,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Graceland,3lbRocWWpBVO9eefdjGi60,spotify:track:3lbRocWWpBVO9eefdjGi60,12,2015-09-16T22:33:32Z,Graceland (2011 Remaster),4O7d2vnCa5sfyZph6gU4sa,spotify:album:4O7d2vnCa5sfyZph6gU4sa,Paul Simon,2CvCyf1gEVhI0mX6aFXmVI,...,0.690819,4.685153e-04,1,0.079664,-9.653,0,0.033910,117.932,4,0.868158
1,Speak In,76GGyeUiOtzsZjCxIJGDpd,spotify:track:76GGyeUiOtzsZjCxIJGDpd,0,2015-09-16T22:33:32Z,It Could Just Be This Place,14M1Is6t3MOkxjBynarKce,spotify:album:14M1Is6t3MOkxjBynarKce,not an Airplane,2eTQ0GfNG2zrAghLUKkSas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,The Funeral,4o0NjemqhmsYLIMwlcosvW,spotify:track:4o0NjemqhmsYLIMwlcosvW,76,2015-09-16T22:33:32Z,Everything All The Time,5uMfshtC2Jwqui0NUyUYIL,spotify:album:5uMfshtC2Jwqui0NUyUYIL,Band of Horses,0OdUWJ0sBjDrqHygGUXeCF,...,0.768954,7.311096e-08,0,0.092790,-5.283,0,0.042186,122.507,4,0.111796
3,Open Season,6XPC7ZNTJcnWzQaLZYe3od,spotify:track:6XPC7ZNTJcnWzQaLZYe3od,43,2015-09-16T22:33:32Z,Open Season,3vyA5G1a5ISREXX0A4D2Iv,spotify:album:3vyA5G1a5ISREXX0A4D2Iv,High Highs,1cXVTtkpqSXVhyD32f9MS4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Artificial Light,2Y93hToFlR6CBItrUqsGqG,spotify:track:2Y93hToFlR6CBItrUqsGqG,40,2015-09-16T22:33:32Z,White Lighter,4cPNeJTCwhYoJJ8sfqLOCB,spotify:album:4cPNeJTCwhYoJJ8sfqLOCB,Typhoon,6p6WlxIkeWH8yOYvciFfVl,...,0.536411,2.085726e-01,2,0.357705,-8.108,1,0.044852,89.811,4,0.358969


True

Todo
*  So I'm basically up and running now with being able to play songs.  I would like to go through some of the categories and explore some of the most/least acoustic, live, etc, etc. attributes and display dataframes showing these lists.

*  I want to do some visualization of songs that I've saved on spotify.  What attributes do I tend to dig? Is there anything with a really narrow variance?  To do this, I need to download my spotify playlists and do some exploration with histograms, bar graphs, etc.

*  I'm going to start getting into linear regression and shit, possibly alongside the textbook.

##Conclusions:

* Radiostations should change according to your emotions.  That's why internet radio always gets old.
* why isn't my listen count available?